In [48]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import regex as re
from bs4 import BeautifulSoup
import requests
import urllib
from urllib.request import urlopen
from selenium.common.exceptions import NoSuchElementException
from collections import Counter
import math
import io
from PyPDF2 import PdfReader
import datefinder
import plotly.express as px
import plotly.graph_objects as go

In [85]:
data = pd.read_csv('PlotData.csv')
data = data.rename(columns={'Doc_Length': "Pagina's"})
old_data = data.copy()
data = data[data['dagen'] > 0]
data_wrong = old_data[(old_data['dagen'] < 1) | (old_data['dagen'].isna())]

data = data_correct.copy()
print(len(old_data))
print(len(data))
print(len(data_wrong))

1112
921
191


In [86]:
def check_timeframes(timeframes):
    
    Correct_Date = True
    
    for timeframe in timeframes:
        if len(timeframe) != 16:
            Correct_Date = False       
        if timeframe[4] != ' ':
            Correct_Date = False
        if timeframe[-10:] != 'e kwartaal':
            Correct_Date = False
        if not re.findall('(:19[7-9]\d|2\d{3})(?=\D|$)', timeframe[0:4]):
            Correct_Date = False
        if not timeframe[5] in ['1','2','3','4']:
            Correct_Date = False
            
    return Correct_Date

def data_subset(data, start_time, end_time):  
    
    data['kwartaal_int'] = data['kwartaal'].astype(str).str.slice(0 , 1)

    cols = data.columns

    min_jaar = int(start_time[0:4])
    max_jaar = int(end_time[0:4])

    min_kwartaal = start_time[5]
    max_kwartaal = end_time[5]

    rows = []

    for i, r in data.iterrows():

        if r.jaar < min_jaar or r.jaar > max_jaar:
            pass

        elif min_jaar == max_jaar:
            if r.kwartaal_int >= min_kwartaal and r.kwartaal_int <= max_kwartaal:
                rows.append(list(r))

        else:
            if r.jaar == min_jaar:
                if min_kwartaal > r.kwartaal_int:
                    pass
                else:
                    rows.append(list(r))
            if r.jaar == max_jaar:
                if max_kwartaal < r.kwartaal_int:
                    pass
                else:
                    rows.append(list(r))        

    data = pd.DataFrame(data = rows, columns = cols)
    data = data.drop('kwartaal_int', axis=1)

    return data

In [89]:
# hier komen de functies voor het maken van alle plots!
def avg_doc_len_ministerie(data):
    avg_doc_length = data.groupby('Ministerie')["Pagina's"].mean().reset_index()
    fig = px.bar(avg_doc_length, x='Ministerie', y="Pagina's", 
                 title="Gemiddeld aantal vrijgegeven pagina's per verzoek voor elke ministerie", color='Ministerie')
    
    fig.update_traces(hovertemplate='%{y:.0f}')
    return fig

def verzoeken_per_ministerie(data):
    requests_by_ministry = data.groupby("Ministerie").size().reset_index(name="Aantal Verzoeken")
    fig1 = px.bar(requests_by_ministry, x="Ministerie", y="Aantal Verzoeken", color='Ministerie')
    fig1.update_traces(hovertemplate='%{y:.0f}')
    fig1.show()
    
def scatter(data):
    fig2 = px.scatter(data, x="dagen", y="Pagina's", color="Ministerie")
    fig2.show()

scatter(data)

In [55]:
def maak_plots(start_time = '2022 1e Kwartaal', end_time = '2023 1e Kwartaal', data = data):
    
    timeframes = [start_time.lower(), end_time.lower()]
    
    if check_timeframes(timeframes) == False:
        data = data
    
    else:
        data = data_subset(data, start_time, end_time)

    display(data)
        
    ## TODO maken van plots hier
        
    
print(maak_plots('2022 1e Kwartaal', '2023 1e Kwartaal'))

,Verzoek,docdatums,urls,verdatums,verantwoordelijke,jaar,kwartaal,image,verdatum,OCR_datum,Doc_Length,final_date,dagen,Publicatiedatum,Ministerie
0,Beslissing op bezwaar Wob verzoek corresponden...,2022-09-15,['https://open.overheid.nl/documenten/ronl-34e...,"[nan, nan, nan]",Ministerie van Infrastructuur en Waterstaat,2022,3e Kwartaal,False,NaN,NaN,20,NaN,NaN,06-10-2022,I&W
1,Beslissing op bezwaar Wob-/Woo-verzoek over in...,2022-12-23,['https://open.overheid.nl/documenten/ronl-7a0...,"['14-07-2022', nan, nan]",Ministerie van Infrastructuur en Waterstaat,2022,4e Kwartaal,False,14-07-2022,NaN,185,2022-07-14,162.0,11-01-2023,I&W
2,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,2023-01-27,['https://open.overheid.nl/documenten/ronl-0ba...,"['image', '02-11-2021', '02-11-2021']","Ministerie van Landbouw, Natuur en Voedselkwal...",2023,1e Kwartaal,True,02-11-2021,NaN,79,2021-11-02,451.0,27-02-2023,LNV
3,Beslissing op bezwaar Wob-besluit Financiën ov...,2022-04-26,['https://open.overheid.nl/documenten/ronl-e3e...,"['22-10-2021', nan]",Ministerie van Financiën,2022,2e Kwartaal,False,22-10-2021,NaN,5,2021-10-22,186.0,16-05-2022,FIN
4,Beslissing op bezwaar Wob-besluit over afstemm...,2023-02-09,['https://open.overheid.nl/documenten/ronl-07e...,"[nan, '05-10-2022']",Ministerie van Financiën,2023,1e Kwartaal,False,05-10-2022,NaN,5,2022-10-05,127.0,09-02-2023,FIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107,Besluit van 20 juli 2022 op het verzoek inzake...,2022-07-20,['https://open.overheid.nl/documenten/ronl-751...,['image'],Ministerie van Justitie en Veiligheid,2022,3e Kwartaal,True,NaN,22-03-2022,69,2022-03-22,120.0,20-07-2022,J&V
1108,Woo-verzoek Evaluatiecommissie Tuitjenhorn,2022-06-30,['https://open.overheid.nl/documenten/ronl-370...,"['22-10-2021', nan]",Ministerie van Justitie en Veiligheid,2022,2e Kwartaal,False,22-10-2021,NaN,357,2021-10-22,251.0,30-06-2022,J&V
1109,Woo-verzoek over de identiteit achter anonieme...,2022-08-25,['https://open.overheid.nl/documenten/ronl-2a0...,['02-08-2022'],"Ministerie van Landbouw, Natuur en Voedselkwal...",2022,3e Kwartaal,False,02-08-2022,NaN,3,2022-08-02,23.0,25-08-2022,LNV
1110,Woo-verzoek over landbouwvrijstelling in inkom...,2023-02-27,['https://open.overheid.nl/documenten/ronl-1c0...,"[nan, '07-08-2022', nan]",Ministerie van Financiën,2023,1e Kwartaal,False,07-08-2022,NaN,15,2022-08-07,204.0,27-02-2023,FIN


None
